In [1]:
# libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service

In [12]:
s = Service(r"C:\Users\Hebert Palmera\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

driver = webdriver.Chrome(service=s)

driver.get("https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3822150905&f_SB2=1&f_TPR=a1705895389-&f_WT=1%2C2%2C3&geoId=106981407&keywords=operations%20analyst&location=Ohio%2C%20United%20States&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R")

jobs_listing = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')

for listing in jobs_listing:
    job_title_element = listing.find_element(By.CLASS_NAME, 'base-search-card__title')
    job_title = job_title_element.text

    company_element = listing.find_element(By.CLASS_NAME, 'base-search-card__subtitle')
    company = company_element.text

    print("Job Title:", job_title)
    print("Company:", company)
    print("-" * 50)

Job Title: Transportation/I-Trade Operations Analyst
Company: Procter & Gamble
--------------------------------------------------
Job Title: Operations Manager - Manufacturing
Company: CyberCoders
--------------------------------------------------
Job Title: Operations Manager - Manufacturing
Company: CyberCoders
--------------------------------------------------
Job Title: SI&OP Senior Analyst
Company: The AZEK Company
--------------------------------------------------
Job Title: Sales Operations Analyst
Company: Lakeview Farms
--------------------------------------------------
Job Title: SQL Reporting Analyst
Company: Watson's
--------------------------------------------------
Job Title: Operations Manager
Company: Reliance One, Inc.
--------------------------------------------------
Job Title: Business Process Analyst
Company: Insight Global
--------------------------------------------------
Job Title: Business Analyst
Company: Innova Solutions
--------------------------------------